### American Sign Language - Transfer Learning

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import Adam

### Cargar Imagenes de Entrenamiento y Prueba

In [ ]:
#https://www.kaggle.com/grassknoted/asl-alphabet

ENTRENAMIENTO_DIR = r"C:\Users\XPC\Desktop\Datasets\ASL\asl_alphabet_train"

IMG_SIZE = 200
BATCH_SIZE = 32
NUM_CLASES = 29

# configuracion de entrenamiento
entrenamiento_datagen = ImageDataGenerator(
    rescale = 1./255, 
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1, 
    shear_range=0.1, 
    zoom_range=0.1,
    horizontal_flip=True,
    validation_split=0.2)

# generador
generador_entrenamiento = entrenamiento_datagen.flow_from_directory(
    ENTRENAMIENTO_DIR,
    target_size=(IMG_SIZE,IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

# configuacion de pruebas (test set)
pruebas_datagen = ImageDataGenerator(rescale=1./255)

# generador ()
generator_pruebas = entrenamiento_datagen.flow_from_directory(
        ENTRENAMIENTO_DIR,
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        subset='validation'
)

In [ ]:
# iterator
aug_iter = entrenamiento_datagen.flow_from_directory(
    ENTRENAMIENTO_DIR,
    target_size=(IMG_SIZE,IMG_SIZE),
    batch_size=1,
    class_mode='categorical',
    subset='training'
)

# mostrar imagenes horizontales
fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(15,15))

# obtiene los labels de las clases de flow from directory
def getLabel(index):
    return (list(aug_iter.class_indices.keys())[list(aug_iter.class_indices.values()).index(index)])

# mostrar 3 imagenes
for i in range(3):
    
    obj = next(aug_iter)
    
    # obtenemos el index de cada clase
    class_index = np.argmax(obj[1])
    
    # obtener imagenes del generador
    image = obj[0]

    # mostar 3 imagenes
    ax[i].imshow(image.reshape(IMG_SIZE, IMG_SIZE, 3))
    ax[i].set_title(getLabel(class_index))
    ax[i].axis('off')

### Transfer Learning VGG16

https://keras.io/api/applications/

In [ ]:
inputs = keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))

# === InceptionV3 CNN Model ===================
base_model = keras.applications.VGG16(
    weights = 'imagenet',
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False)

base_model.trainable = False
# === InceptionV3 CNN Model ===================

# === Arquitectura ===================
x = base_model.output
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(1024, activation='relu')(x)
x = keras.layers.Dropout(0.2)(x)
outputs = keras.layers.Dense(NUM_CLASES, activation='softmax')(x)

# se acopla el modelo
model = keras.Model(base_model.input, outputs)
# === Arquitectura ===================

# congelar capas
for layer in base_model.layers:
    layer.trainable = False

# compilar el modelo.
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.001),
              metrics=['accuracy'])

model.summary()

### Inicio del Entrenamiento

In [ ]:
history = model.fit(
    generador_entrenamiento,
    steps_per_epoch=80,  
    epochs=10,
    verbose=1,
    validation_data=generator_pruebas,
    validation_steps=50
)

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.ylim([0, 1.1])
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.ylim([0, 1.1])
plt.show()

## Realizar algunas predicciones!

In [ ]:
from os import listdir
from os.path import isfile, join
from keras.preprocessing import image

fig, ax = plt.subplots(nrows=10, ncols=3, figsize=(15,15))

# obtiene los labels de las clases de flow from directory
def getLabel(index):
    return (list(aug_iter.class_indices.keys())[list(aug_iter.class_indices.values()).index(index)])

TEST_DIR = r"C:\Users\XPC\Desktop\Datasets\ASL\asl_alphabet_test"

files = [f for f in listdir(TEST_DIR) if isfile(join(TEST_DIR, f))]
n = len(files)
c = 0
ind = 0
for i in range(10):
    for j in range(3):
        
        if (i == 9 and j > 0):
            break
        
        label = files[ind].split("_")[0]
        url = TEST_DIR + "/" + files[ind]
        img = image.load_img(url, target_size=(IMG_SIZE, IMG_SIZE,3), interpolation="nearest")
        x = image.img_to_array(img) / 255.
        x = np.expand_dims(x, axis=0)
        # ======= Pred ======= 
        pred = getLabel(np.argmax(model.predict([x])))
        if (label == pred):
            c = c + 1
        # ======= Pred ======= 
        ax[i][j].imshow(img)
        ax[i][j].set_title(label+"-"+pred)
        ax[i][j].axis('off')
        ind = ind + 1

print("Test Acc:",(c/n))

In [ ]:
# iterator
aug_iter = entrenamiento_datagen.flow_from_directory(
        ENTRENAMIENTO_DIR,
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=1,
        class_mode='categorical',
        subset='validation'
)

# mostrar imagenes horizontales
fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(15,15))

# obtiene los labels de las clases de flow from directory
def getLabel(index):
    return (list(aug_iter.class_indices.keys())[list(aug_iter.class_indices.values()).index(index)])

# mostrar 3 imagenes
for i in range(3):

    obj = next(aug_iter)
    
    # obtenemos el index de cada clase
    class_index = np.argmax(obj[1])
    
    # obtener imagenes del generador
    image = obj[0]
    
    # valor real
    real = getLabel(class_index)
    # prediccion!
    pred = getLabel(np.argmax(model.predict(image)))
    
    # mostar 3 imagenes
    ax[i].imshow(image.reshape(IMG_SIZE, IMG_SIZE, 3))
    ax[i].set_title(real+"\n\n"+pred+"\n")
    ax[i].axis('off')